In [ ]:
#create the dynamodb table
import boto3



# Create a Boto3 session
session = boto3.Session(
    aws_access_key_id='YOUR_ACCESS_KEY',     # Replace with your actual access key
    aws_secret_access_key='YOUR_SECRET_KEY', # Replace with your actual secret key
    region_name='YOUR_REGION'                 # Replace with your desired region
)

# Use the session to create a DynamoDB resource
dynamodb = session.resource('dynamodb')

# Example: List all tables in DynamoDB
tables = list(dynamodb.tables.all())
print("Existing tables:", tables)

# Example: Create a DynamoDB table using the session
table_name = 'post_table'
table = dynamodb.create_table(
    TableName=table_name,
    KeySchema=[
        {
            'AttributeName': 'PostID',
            'KeyType': 'HASH'  # Partition key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'PostID',
            'AttributeType': 'S'  # S for string
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

# Wait until the table exists and is active
table.meta.client.get_waiter('table_exists').wait(TableName=table_name)

print(f"Table {table_name} created successfully.")

In [ ]:
# lambda function for post the hashtags

import json
import boto3
from datetime import datetime
import uuid

# Initialize DynamoDB resource
dynamodb = boto3.resource('dynamodb')

def lambda_handler(event, context):
    # Extract post data from the event
    try:
        post_text = event['post_text']
        post_hashtags = event.get('post_hashtags', [])
    except KeyError as e:
        return {
            'statusCode': 400,
            'body': json.dumps(f"Missing key: {str(e)}")
        }

    # Create a unique post ID
    post_id = str(uuid.uuid4())

    # Get the current timestamp
    timestamp = datetime.utcnow().isoformat()

    # Specify the DynamoDB table
    table = dynamodb.Table('post_table')

    # Build the item to store in the table
    item = {
        'PostID': post_id,
        'post_text': post_text,
        'post_hashtags': post_hashtags,
        'timestamp': timestamp
    }

    # Store the item in the DynamoDB table
    try:
        table.put_item(Item=item)
        return {
            'statusCode': 200,
            'body': json.dumps('Post successfully stored!')
        }
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps(f"Error storing post: {str(e)}")
        }

In [ ]:
# lambda function fetching the Top 3 trending hashtags

import boto3
from collections import Counter


def fetch_trending_hashtags():
    dynamodb = boto3.resource('dynamodb', region_name='eu-north-1')
    table = dynamodb.Table('post_table')

    # Scan the table to retrieve all posts
    response = table.scan()
    items = response['Items']

    # Extract hashtags from each post and count them
    all_hashtags = []
    for item in items:
        hashtags = item.get('post_hashtags', [])  # Use .get() to avoid KeyError
        if isinstance(hashtags, list):  # Ensure hashtags is a list
            all_hashtags.extend(hashtags)

    # Count the occurrence of each hashtag
    hashtag_counts = Counter(all_hashtags)

    # Sort hashtags by count
    trending_hashtags = hashtag_counts.most_common(4)  # Fetch top 4 trending hashtags

    return trending_hashtags

def lambda_handler(event, context):
    trending_hashtags = fetch_trending_hashtags()
    return {
        'statusCode': 200,
        'body': trending_hashtags
    }